In [37]:
import gurobipy as gp
from gurobipy import GRB, quicksum, multidict, tuplelist, tupledict

from reader import read
from Pizza import Pizza

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
def transform_solution(X_i, ING, ingredients):
    chosen_ingredients = []
    for i in ING:
        if X_i[i].x == 1:
            chosen_ingredients.append(ingredients[i])
            
    return chosen_ingredients

In [50]:
fname = "d_difficult.in.txt"

clients, ingredients = read(f'data/{fname}')

nb_clients, nb_ing = len(clients), len(ingredients)
CLI = range(nb_clients)
ING = range(nb_ing)

In [55]:
nb_clients

9368

In [51]:
model = gp.Model("pizza")

X_i = model.addVars(nb_ing, vtype=GRB.BINARY,  name="X")
C_j = model.addVars(nb_clients, vtype=GRB.BINARY,  name="C")

for j, c in enumerate(clients):
    const = model.addConstr( quicksum( X_i[i] for i in c[0]) >= \
                               len(c[0]) * C_j[j], name=f'LIKE[{j}]')
    const = model.addConstr( quicksum( X_i[i] for i in c[1]) <= \
                               len(c[1]) * (1 - C_j[j]), name=f'DISLIKE[{j}]')

model.setObjective( quicksum(C_j[j] for j in CLI), sense = GRB.MAXIMIZE )

In [52]:
model.update()
model.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 18736 rows, 9968 columns and 49281 nonzeros
Model fingerprint: 0xd59a7371
Variable types: 0 continuous, 9968 integer (9968 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 1420.0000000
Presolve removed 3751 rows and 990 columns
Presolve time: 0.16s
Presolved: 14985 rows, 8978 columns, 44661 nonzeros
Variable types: 0 continuous, 8978 integer (8978 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   17651    4.8587444e+03   8.540586e+03   0.000000e+00      5s
   23619    4.6840000e+03   0.000000e+00   0.000000e+00     10s

Root relaxation: objective 4.684000e+03, 23619 iterations, 9.84 seco

H  267   274                    1804.0000000 2426.16691  34.5%  5463 1092s
H  270   274                    1805.0000000 2426.16691  34.4%  5429 1092s
   273   295 2276.63099   44 5899 1805.00000 2426.16691  34.4%  5415 1101s
   294   316 2263.78069   47 5797 1805.00000 2426.16691  34.4%  5255 1113s
   315   323 2254.16188   51 5761 1805.00000 2426.16691  34.4%  5119 1122s
   322   342 2251.03024   52 5701 1805.00000 2426.16691  34.4%  5070 1140s
   341   372 2243.22090   54 5638 1805.00000 2426.16691  34.4%  4946 1151s
   371   378 2233.73011   57 5591 1805.00000 2426.16691  34.4%  4748 1183s
   377   411 2231.32156   58 5591 1805.00000 2426.16691  34.4%  4719 1194s
   410   417 2217.53770   62 5520 1805.00000 2426.16691  34.4%  4540 1224s
   416   455 2216.45735   63 5492 1805.00000 2426.16691  34.4%  4508 1235s
   454   496 2197.45998   68 5343 1805.00000 2426.16691  34.4%  4316 1247s
   495   547 2178.21847   72 5244 1805.00000 2426.16691  34.4%  4126 1261s
   546   609 2162.48971  

In [53]:
chosen_ingredients = transform_solution(X_i, ING, ingredients)

In [54]:
my_pizza = Pizza(chosen_ingredients)
my_pizza.write_sol(f'output/{fname}')